In [ ]:
#import libraires

import os
import numpy as np
import argparse 
from plantcv import plantcv as pcv

In [ ]:
class options:
    def __init__(self):
        self.image = "image_example/raspiY_side3_2021.10.15-13.43.24.jpg"
        self.debug = "plot"
        self.writeimg= True
        self.result = "./Results"
        self.outdir = "./Results"
        
# Store the output to the current directory 
# Get options
args = options()

# Set debug to the global parameter 
pcv.params.debug = args.debug


In [ ]:
# Read image

# Inputs:
#   filename - Image file to be read in 
#   mode - How to read in the image; either 'native' (default), 'rgb', 'gray', or 'csv'
img, path, filename = pcv.readimage(filename=args.image)

In [ ]:
img1 = pcv.white_balance(img, roi=(995,400,55,55))

In [ ]:
#Rotate the image slightly so the plants line up with
#the grid that we'll add in a later step
#Inputs:
#img = image object, RGB color space¶
#rotation_deg = Rotation angle in degrees, can be negative, positive values
#will move counter-clockwise
#crop = If True then image will be cropped to orginal image dimensions, if False
#the image size will be adjusted to accommodate new image dimensions
img1 = pcv.rotate(img=img1, rotation_deg=0, crop=False)

In [ ]:
# Shift image. This step is important for clustering later on.
# For this image it also allows you to push the green raspberry pi camera
# out of the image. This step might not be necessary for all images.
# The resulting image is the same size as the original.

# Inputs:
#   img    = image object
#   number = integer, number of pixels to move image
#   side   = direction to move from "top", "bottom", "right","left"
imgs = pcv.shift_img(img=img1, number=20, side='left')
img1 = pcv.shift_img(img=imgs, number=20, side='top')

In [ ]:
# Convert image from RGB color space to LAB color space
# Keep only the green-magenta channel (grayscale)

# Inputs:
#    rgb_img = image object, RGB color space
#    channel = color subchannel ('l' = lightness, 'a' = green-magenta , 'b' = blue-yellow)
a = pcv.rgb2gray_lab(rgb_img=img1, channel='a')

In [ ]:
np.min(a)
np.max(a)

In [ ]:
# Set a binary threshold on the saturation channel image

# Inputs:
#    gray_img    = img object, grayscale
#    threshold   = threshold value (0-255)
#    max_value   = value to apply above threshold (usually 255 = white)
#    object_type = light or dark
#       - If object is light then standard thresholding is done
#       - If object is dark then inverse thresholding is done
img_binary = pcv.threshold.binary(gray_img=a, threshold=104, max_value=255, object_type='dark')
#                                                        ^
#                                                        |
#                                         adjust this value

In [ ]:
id_objects, obj_hierarchy = pcv.find_objects(img=img1, mask= img_binary)

In [ ]:
# Define region of interest (ROI)

# Inputs:
#    img   = An RGB or grayscale image to plot the ROI on.
#    x     = The x-coordinate of the upper left corner of the rectangle.
#    y     = The y-coordinate of the upper left corner of the rectangle.
#    h     = The width of the rectangle.
#    w     = The height of the rectangle.
#   roi_contour, roi_hierarchy = pcv.roi.rectangle(600, 900, 20000, 20000, img1) 
#                                                  |______________|
#                                            adjust these four values
roi_contour, roi_hierarchy = pcv.roi.rectangle(img=img1, x=75, y=95, h=1875, w=1970)


In [ ]:
# Keep objects that overlap with the ROI

# Inputs:
#    img            = img to display kept objects
#    roi_contour    = contour of roi, output from any ROI function
#    roi_hierarchy  = contour of roi, output from any ROI function
#    object_contour = contours of objects, output from pcv.find_objects function
#    obj_hierarchy  = hierarchy of objects, output from pcv.find_objects function
#    roi_type       = 'partial' (default, for partially inside the ROI), 'cutto', or 
#                     'largest' (keep only largest contour)
roi_objects, roi_obj_hierarchy, kept_mask, obj_area = pcv.roi_objects(img=img1, roi_contour=roi_contour, 
                                                                      roi_hierarchy=roi_hierarchy,
                                                                      object_contour=id_objects, 
                                                                      obj_hierarchy=obj_hierarchy, 
                                                                      roi_type='partial')

In [ ]:
# Check if all of the plants fall completely within the bounds of an image 
# or if it touches the edge. Used for QC. 

# Inputs:
#   mask = Binary mask 
in_bounds = pcv.within_frame(mask=kept_mask)

In [ ]:
# This function take a image with multiple contours and
# clusters them based on user input of rows and columns

# Inputs:
#    img               = An RGB or grayscale image
#    roi_objects       = object contours in an image that are needed to be clustered.
#    roi_obj_hierarchy = object hierarchy
#    nrow              = number of rows to cluster (this should be the approximate  number of 
#                        desired rows in the entire image even if there isn't a literal row of plants)
#    ncol              = number of columns to cluster (this should be the approximate number of 
#                        desired columns in the entire image even if there isn't a literal row of plants)
#    show_grid         = if True then the grid is drawn on the image, default show_grid=False
clusters_i, contours, hierarchies = pcv.cluster_contours(img=img1, roi_objects=roi_objects, 
                                                         roi_obj_hierarchy=roi_obj_hierarchy, 
                                                         nrow=4, ncol=4, 
                                                         show_grid=True)


In [ ]:
rois1, roi_hierarchy1 = pcv.roi.multi(img=img1, coord=(330,250), radius=180, spacing=(475, 475), nrows=4, ncols=4)

In [ ]:
img_copy = np.copy(img1)
for i in range(0, len(rois1)):
    roi = rois1[i]
    hierarchy = roi_hierarchy1[i]
    # Filter objects by ROI 
    filtered_contours, filtered_hierarchy, filtered_mask, filtered_area = pcv.roi_objects(img=img_copy, roi_type="partial", roi_contour=roi, roi_hierarchy=hierarchy, object_contour=roi_objects, obj_hierarchy=roi_obj_hierarchy)

    # Combine objects together in each plant     
    plant_contour, plant_mask = pcv.object_composition(img=img_copy, contours=filtered_contours, hierarchy=filtered_hierarchy)         
        
    #Build a new array to distinguish missing plants
    test = np.array(plant_contour)
        
     # Analyze the shape of each existing plant:
    if test.dtype == 'int32':
        analysis_images = pcv.analyze_object(img=img_copy, obj=plant_contour, mask=plant_mask)
            
        # Save the image with shape characteristics 
        test2 = np.array(analysis_images)
        if test2.dtype == 'uint8':
            img_copy = analysis_images
                
            pcv.outputs.add_observation(sample = 'default',variable = 'plantID', trait = 'roi', method = 'roi', scale = 'int', datatype = int, value= filename[:-4] + "-" + str(i), label = '#')

            # Print out a text file with shape data for each plant in the image 
            pcv.print_results(filename = args.result + str(i) + '.txt')
            # Clear the measurements stored globally into the Ouptuts class
            pcv.outputs.clear()  


if args.writeimg:
    outfile = os.path.join(args.outdir, filename[:-4] + ".png")
    pcv.print_image(img=img_copy, filename=outfile)

